In [11]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import time
import tensorflow as tf

# generate artificial sine signal data
# x_values are the processs data (float32)
x_values = np.random.uniform(low=0, high=2*math.pi, size= 1000).astype(np.float32)
np.random.shuffle(x_values)
# y_values are the data labels (float32)
y_values = np.sin(x_values.astype(np.float32))

# split the data into training, validation, and testing
samples = 1000
train_split = int(0.6*samples)
test_split = int(0.2*samples + train_split)
x_train, x_valid, x_test = np.split (x_values, [train_split, test_split])
y_train, y_valid, y_test = np.split (y_values, [train_split, test_split])

# normalize the data by subtrating the mean and dividing by the standard
# deviation (-1 and +1)
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train -= mean
x_train /= std
x_valid -= mean
x_valid /= std
x_test -= mean
x_test /= std

# export the test data to a text file
np.savetxt('x_test_normalized.txt', x_test, fmt='%.6f', delimiter=',')
np.savetxt('y_test.txt', y_test, fmt='%.6f', delimiter=',')

# define the model
model = tf.keras.Sequential()

# add layers to the model (input 16, middle 16, last 1 (outputting a number))
model.add(tf.keras.layers.Dense(16, activation= 'relu', input_shape=(1, )))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

# compile the model (optimizer function is stochastic gradient descent, and loss
# is MSE)
model.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])
# optimizer = tf.keras.optimizers.sgd(learning_rate = 0.001)

# train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_valid, y_valid), verbose=2)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range (1, len(loss) + 1)

# plt.plot(epochs, loss, 'g', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='validation_loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# one way to test the model
model.evaluate(x_test, y_test)

# another way to test the model using individual data points (test normalized)
prediction = model.predict (x_test)
print (prediction)

# plt.clf()
# plt.title('Test data and predicted values')
# plt.scatter (x_test, y_test)
# plt.scatter (x_test, prediction)
# plt.show ()

# convert the model to tflite (float)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
regressionfloat = converter.convert()

# save the converted model as a file
open("regressionfloat", "wb").write(regressionfloat)

# export the converted float model to a header file
!xxd -i regressionfloat > regressionfloat.h

# convert the model to int8 (quantization)

# quantize the model to int8 and then convert and export

# we need a representative dataset function
def representative_dataset (num_samples=500):
  for i in range (num_samples):
    yield[x_values[i].reshape(1, 1)]

# convert the model to tflite (int8)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset
regressionint8 = converter.convert()

# save the converted model
open("regressionint8", "wb").write(regressionint8)

# export the converted int8 model to a header file
!xxd -i regressionint8 > regressionint8.h

## make sure to change the name of the exported header file to model_data.h
## inside the header, define the array as: const unsigned char g_model[]
## in the bottom of the file, define the length as: unsigned int g_model_len =


Epoch 1/50
10/10 - 1s - loss: 0.5269 - mae: 0.6605 - val_loss: 0.3918 - val_mae: 0.5575 - 1s/epoch - 107ms/step
Epoch 2/50
10/10 - 0s - loss: 0.3646 - mae: 0.5380 - val_loss: 0.3052 - val_mae: 0.4809 - 211ms/epoch - 21ms/step
Epoch 3/50
10/10 - 0s - loss: 0.2938 - mae: 0.4770 - val_loss: 0.2547 - val_mae: 0.4371 - 101ms/epoch - 10ms/step
Epoch 4/50
10/10 - 0s - loss: 0.2483 - mae: 0.4373 - val_loss: 0.2210 - val_mae: 0.4082 - 79ms/epoch - 8ms/step
Epoch 5/50
10/10 - 0s - loss: 0.2156 - mae: 0.4072 - val_loss: 0.1946 - val_mae: 0.3846 - 76ms/epoch - 8ms/step
Epoch 6/50
10/10 - 0s - loss: 0.1896 - mae: 0.3823 - val_loss: 0.1748 - val_mae: 0.3659 - 88ms/epoch - 9ms/step
Epoch 7/50
10/10 - 0s - loss: 0.1693 - mae: 0.3624 - val_loss: 0.1596 - val_mae: 0.3505 - 79ms/epoch - 8ms/step
Epoch 8/50
10/10 - 0s - loss: 0.1534 - mae: 0.3454 - val_loss: 0.1477 - val_mae: 0.3378 - 83ms/epoch - 8ms/step
Epoch 9/50
10/10 - 0s - loss: 0.1407 - mae: 0.3310 - val_loss: 0.1387 - val_mae: 0.3281 - 82ms/epoch

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
